In [1]:
import numpy as np
import pandas as pd
import trimesh
import slam.io as sio
import nibabel as nib
from mne import read_epochs
from mne.channels import read_layout
from freesurfer_tools import transform_atlas, fsavg_vals_to_native
import matplotlib
matplotlib.use("Qt5Agg")
from matplotlib import cm, colors
import matplotlib.pylab as plt
import new_files
files = new_files.Files()
from sklearn.preprocessing import minmax_scale
from scipy.interpolate import interp2d, interp1d
from scipy.spatial.distance import euclidean
from scipy.signal import savgol_filter

In [2]:
def compute_csd(surf_tcs, times, mean_dist, n_surfs):
    # Compute CSD
    nd=1;
    spacing=mean_dist*10**-3

    csd=np.zeros((n_surfs, surf_tcs.shape[1]))
    for t in range(surf_tcs.shape[1]):
        phi=surf_tcs[:,t]
        csd[0,t]=surf_tcs[0,t]
        csd[1,t]=surf_tcs[1,t]
        for z in range(2,n_surfs-3):
            csd[z,t]=(phi[z+2]-2*phi[z]+phi[z-2])/((nd*spacing)**2)
        csd[-2,t]=surf_tcs[-2,t]
        csd[-1,t]=surf_tcs[-1,t]            
    
    return csd

def csd_layer(data, mean_distance, n_surfs, nd=1):
    spacing = mean_distance*10*1e-3
    output_shape = data.shape
    csd = np.zeros(output_shape)
    for t in range(output_shape[1]):
        phi = data[:, t]
        csd[0, t] = data[0, t]
        csd[1, t] = data[1, t]
        for z in range(2, n_surfs-3):
            csd[z, t] = (phi[z+2]-2*phi[z-2])/((nd*spacing)**2)
        csd[-2, t] = data[-2, t]
        csd[-1, t] = data[-1, t]  
    return csd


def smooth_csd(csd, n_surfs):
    # interpolate CSD in space
    y = np.linspace(0,n_surfs-1,n_surfs)
    Yi=np.linspace(0,n_surfs-1,500)
    
    f=interp1d(y,csd,kind='cubic',axis=0)
    csd_smooth=f(Yi)
    
    csd_smooth=savgol_filter(csd_smooth, 51, 3, axis=1)
    
    return csd_smooth

In [3]:
def get_necessary_files(subject_id, dataset_path, *kwargs):
    all_mu_files = files.get_files(dataset_path, "*.tsv", strings=[subject_id, *kwargs], prefix="multilayer_MU")
    all_epoch_files = files.get_files(dataset_path, "*.fif", strings=[subject_id, "epo", *kwargs], prefix="autoreject")
    
    mu = [i.split("/")[-1] for i in all_mu_files]
    epo = [i.split("/")[-1] for i in all_epoch_files]
    mapx = [any([e[:-4] in m[:-4] for m in mu]) for e in epo]
    all_epoch_files = np.array(all_epoch_files)[mapx].tolist()
    
    all_behav_files = files.get_files(dataset_path, "*.csv", strings=[subject_id, "beh", *kwargs], prefix="autoreject")
    all_pial_path = files.get_files(dataset_path, "*.gii")
    fsnat_sphere_lh = [i for i in all_pial_path if files.check_many([subject_id, "lh.sphere.reg.gii"], i, func="all")][0]
    fsnat_sphere_rh = [i for i in all_pial_path if files.check_many([subject_id, "rh.sphere.reg.gii"], i, func="all")][0]
    fsnat_sphere_paths = [fsnat_sphere_lh, fsnat_sphere_rh]
    pial_path = [i for i in all_pial_path if files.check_many([subject_id, "/pial.gii"], i, func="all")][0]
    pial_ds_path = [i for i in all_pial_path if files.check_many([subject_id, "/pial.ds.gii"], i, func="all")][0]
    pial_ds_nodeep = [i for i in all_pial_path if files.check_many([subject_id, "pial.ds.link_vector.nodeep.gii"], i, func="all")][0]
    white_ds_nodeep = [i for i in all_pial_path if files.check_many([subject_id, "white.ds.link_vector.nodeep.gii"], i, func="all")][0]
    pial_ds_inflated = [i for i in all_pial_path if files.check_many([subject_id, "pial.ds.inflated.nodeep.gii"], i, func="all")][0]
    return all_mu_files, all_epoch_files, all_behav_files, fsnat_sphere_paths, pial_path, pial_ds_path, pial_ds_nodeep, white_ds_nodeep, pial_ds_inflated

In [4]:
glasser = "/home/mszul/git/DANC_multilayer_laminar/assets/atlas_glasser_2016.csv"
dataset_path = "/home/common/bonaiuto/cued_action_meg/derivatives/processed/"
fsavg_sphere_paths = [
    "/home/mszul/git/DANC_multilayer_laminar/assets/lh.sphere.reg.gii",
    "/home/mszul/git/DANC_multilayer_laminar/assets/rh.sphere.reg.gii"
]
annot_paths = [
    "/home/mszul/git/DANC_multilayer_laminar/assets/lh.HCPMMP1.annot",
    "/home/mszul/git/DANC_multilayer_laminar/assets/rh.HCPMMP1.annot"
]
thicc_l_paths = files.get_files("/home/mszul/git/DANC_multilayer_laminar/assets/big_brain_layer_thickness", "*.gii", strings=["hemi-L"], prefix="tpl-fsaverage")
thicc_r_paths = files.get_files("/home/mszul/git/DANC_multilayer_laminar/assets/big_brain_layer_thickness", "*.gii", strings=["hemi-R"], prefix="tpl-fsaverage")
thicc_lr_paths = list(zip(thicc_l_paths, thicc_r_paths))

In [5]:
layout = read_layout("/home/mszul/miniconda3/envs/mne/lib/python3.9/site-packages/mne/channels/data/layouts/CTF275.lay")

In [6]:
subjects = ["sub-001", "sub-002"]
filtered = True

In [7]:
atlas = pd.read_csv(glasser)
target_labels = atlas[
    (atlas.PRIMARY_SECTION == 1) | 
    (atlas.USED_LABEL == "L_MT_ROI") | 
    (atlas.USED_LABEL == "R_MT_ROI")
].USED_LABEL.to_list()
print(target_labels)

['L_V1_ROI', 'L_MT_ROI', 'R_V1_ROI', 'R_MT_ROI']


In [8]:
def calculate_brains(subject_id, dataset_path, output={}, ret=False):
    output[subject_id] = {}
    
    [
        all_mu_files, all_epoch_files, all_behav_files, 
        fsnat_sphere_paths, pial_path, pial_ds_path, 
        pial_ds_nodeep, white_ds_nodeep, pial_ds_inflated
    ] = get_necessary_files(subject_id, dataset_path, "visual1")
    
    mesh_colors, lab_col_map = transform_atlas(annot_paths, fsavg_sphere_paths, fsnat_sphere_paths, pial_path, pial_ds_path, pial_ds_nodeep)
    output[subject_id]["mesh_colors"] = mesh_colors
    output[subject_id]["lab_col_map"] = lab_col_map
    ROI_maps = {i: [i == j.decode("utf=8") for j in lab_col_map] for i in target_labels}
    output[subject_id]["ROI_maps"] = ROI_maps
    
    layers_fsaverage_values = {i+1: [nib.load(i).agg_data() for i in thicc_lr_paths[i]] for i in range(len(thicc_lr_paths))}
    layers_fsnative_ds_values = {i: fsavg_vals_to_native(
        layers_fsaverage_values[i],
        fsavg_sphere_paths,
        fsnat_sphere_paths, 
        pial_path, 
        pial_ds_path, 
        pial_ds_nodeep
    ) for i in layers_fsaverage_values.keys()}
    layers = np.array([layers_fsnative_ds_values[i] for i in layers_fsnative_ds_values.keys()])
    overall_thickness = np.sum(layers, axis=0)
    layers_prop_thickness = {i: np.divide(
        layers_fsnative_ds_values[i], 
        overall_thickness, 
        where=overall_thickness != 0
    ) for i in layers_fsnative_ds_values.keys()}
    output[subject_id]["big_brain_prop_thickness"] = layers_prop_thickness
    
    pial_ds_loc = nib.load(pial_ds_nodeep).agg_data()[0]
    white_ds_loc = nib.load(white_ds_nodeep).agg_data()[0]
    distance_ROI = np.array([euclidean(pial_ds_loc[i], white_ds_loc[i]) for i in range(pial_ds_loc.shape[0])])
    output[subject_id]["fsnat_wp_distance"] = distance_ROI
    
    output[subject_id]["pial__ds_inflated_path"] = pial_ds_inflated
    if ret:
        return output

In [9]:
brain_data = {}
for sub in subjects:
    calculate_brains(sub, dataset_path, output=brain_data)

dict_keys(['mesh_colors', 'lab_col_map', 'ROI_maps', 'big_brain_prop_thickness', 'fsnat_wp_distance', 'pial__ds_inflated_path'])

In [12]:
def calculate_epochs(subject_id, dataset_path, ROI_maps, layout, output={}, ret=False):
    output[subject_id] = {}
    [
        all_mu_files, all_epoch_files, all_behav_files, 
        fsnat_sphere_paths, pial_path, pial_ds_path, 
        pial_ds_nodeep, white_ds_nodeep, pial_ds_inflated
    ] = get_necessary_files(subject_id, dataset_path, "visual1")
    
    if filtered:
        mu_files = [i for i in all_mu_files if "_filt_" in i]
    else:
        mu_files = [i for i in all_mu_files if "_nofilt_" in i]
    all_behav_files.sort()
    all_epoch_files.sort()
    mu_files.sort()
    beh_epo = list(zip(all_behav_files, all_epoch_files, mu_files))
    labels = []
    epos = {roi: [] for roi in ROI_maps.keys()}
    
    for beh_path, epo_path, MU_path in beh_epo:
        beh = pd.read_csv(beh_path).trial_coherence.to_list()
        labels.extend(beh)
        epo = read_epochs(epo_path, verbose=False)
        times = epo.times
        epo = epo.pick_types(meg=True, ref_meg=False, misc=False)
        epo_names = epo.ch_names
        epo = epo.get_data()
        # SENS_MAP = [files.check_many(epo_names, ch_n, func="any") for ch_n in layout.names]
        MU = pd.read_csv(MU_path, sep="\t", header=None).to_numpy()
        print(epo.shape, MU.shape, epo_path.split("/")[-1], MU_path.split("/")[-1])
#         # MU = MU[:, SENS_MAP]
        MU = np.split(MU, 11, axis=0)
        MU = MU[::-1]
        for roi in ROI_maps.keys():
            # layers x epochs x vertices x time
            epos[roi].append(np.array([np.array([np.dot(m[ROI_maps[roi]], e) for e in epo]) for m in MU]))
    epos = {roi: np.concatenate(epos[roi], axis=1) for roi in epos.keys()}
    
    output[subject_id]["epoch_labels"] = labels
    output[subject_id]["epochs"] = epos
    if ret:
        return output

In [13]:
epoch_data = {}
for sub in subjects:
    calculate_epochs(sub, dataset_path, brain_data[sub]["ROI_maps"], layout, output=epoch_data)

(177, 275, 601) (353826, 275) autoreject-sub-001-ses-01-001-visual1-epo.fif multilayer_MU_spm-converted_filt_autoreject-sub-001-ses-01-001-visual1-epo.tsv
(175, 275, 601) (353826, 275) autoreject-sub-001-ses-01-002-visual1-epo.fif multilayer_MU_spm-converted_filt_autoreject-sub-001-ses-01-002-visual1-epo.tsv
(177, 275, 601) (353826, 275) autoreject-sub-001-ses-01-003-visual1-epo.fif multilayer_MU_spm-converted_filt_autoreject-sub-001-ses-01-003-visual1-epo.tsv
(178, 273, 601) (303534, 273) autoreject-sub-002-ses-03-001-visual1-epo.fif multilayer_MU_spm-converted_filt_autoreject-sub-002-ses-03-001-visual1-epo.tsv
(178, 273, 601) (303534, 273) autoreject-sub-002-ses-03-002-visual1-epo.fif multilayer_MU_spm-converted_filt_autoreject-sub-002-ses-03-002-visual1-epo.tsv
(178, 273, 601) (303534, 273) autoreject-sub-002-ses-03-003-visual1-epo.fif multilayer_MU_spm-converted_filt_autoreject-sub-002-ses-03-003-visual1-epo.tsv
(175, 273, 601) (303534, 273) autoreject-sub-002-ses-04-001-visual1-ep

In [15]:
len(epoch_data["sub-001"]["epoch_labels"])

529

In [17]:
epoch_data["sub-001"]["epochs"]['L_V1_ROI'].shape

In [19]:
brain_data["sub-002"].keys()

dict_keys(['mesh_colors', 'lab_col_map', 'ROI_maps', 'big_brain_prop_thickness', 'fsnat_wp_distance', 'pial__ds_inflated_path'])

In [ ]:
epochs = {}

for sub in subjects:
    for lab in target_labels:
        ex = {
            "epochs": epoch_data[sub]["epochs"][lab],
            "labels": epoch_data[sub]["epoch_labels"]
        }
        with open('epoch_{}_{}.pickle'.format(sub, lab), 'wb') as f:
            pickle.dump(ex, f)